In [2]:
from flask import Flask, render_template, request
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import load_model

In [3]:
app = Flask("Cow Weight Extractor")

In [4]:
# Load the trained model
model = load_model('cow_weight_model.h5')

In [5]:
# Function to preprocess image and get the predicted weight
def get_predicted_weight(img):
    img = preprocess_image(img)

    # Make a prediction using the loaded model
    prediction = model.predict(np.expand_dims(img, axis=0))[0][0]

    return prediction

In [23]:
# Function to preprocess the image 
import os
import tensorflow as tf

def preprocess_image(img_path):
    # Read the image file
    img = tf.io.read_file(img_path)
    
    # Decode the image to a tensor
    img = tf.image.decode_image(img, channels=3)
    
    # Resize the image to your desired dimensions
    img = tf.image.resize(img, [300, 300])
    
    # Normalize the pixel values to the range [0, 1]
    img = tf.cast(img, tf.float32) / 255.0
    
    return img

def process_images_in_directory(main_directory):
    # Get the list of subdirectories inside the main directory
    subdirectories = [os.path.join(main_directory, d) for d in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, d))]
    
    # Iterate through each subdirectory
    for subdirectory in subdirectories:
        print(f"Processing images in {subdirectory}...")
        
        # Get the list of image files in the subdirectory
        image_files = [os.path.join(subdirectory, f) for f in os.listdir(subdirectory) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
        # Process each image in the subdirectory
        for image_file in image_files:
            preprocessed_img = preprocess_image(image_file)
            
            

# Specify the main directory containing subdirectories of cattle breeds
main_directory = 'Cattle Breeds'

# Call the function to process images in all subdirectories
process_images_in_directory(main_directory)


Processing images in Cattle Breeds\Ayrshire cattle...
Processing images in Cattle Breeds\Brown Swiss cattle...
Processing images in Cattle Breeds\Holstein Friesian cattle...
Processing images in Cattle Breeds\Jersey cattle...
Processing images in Cattle Breeds\Red Dane cattle...


In [19]:
# Route to handle image uploads
if 'predict_weight' in app.view_functions:
    del app.view_functions['predict_weight']

@app.route('/predict_weight', methods=['POST'])
def predict_weight():
    # Get the image from the request
    image = request.files['image'].read()
    nparr = np.fromstring(image, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Get the predicted weight
    predicted_weight = get_predicted_weight(img)

    return str(predicted_weight)

In [13]:
# # Main route to display the camera feed
@app.route('/')
def index():
    return render_template('index.html')

if 'Cow Weight Extractor' == '__main__':
    app.run(debug=True)